<h3>queue</h3>

In [1]:



# ******************************************************************************
#
# This is a discrete-event simulator (event scheduling approach) of a queue
# - single-server 
# - infinite capacity of the waiting line
# - exponential inter arrival times
# - exponential service times
#
# 
# ******************************************************************************
 


import random
from queue import PriorityQueue

# ******************************************************************************
# Constants
# ******************************************************************************

LOAD=0.85  # load of the queue
SERVICE = 10.0 # av service time
ARRIVAL   = SERVICE/LOAD # av. inter-arrival time
TYPE1 = 1 # At the beginning all clients are of the same type, TYPE1 

SIM_TIME = 500000 # condition to stop the simulation


# ******************************************************************************
# To take the measurements
#
# Collect
# - total number of arrivals, Narr
# - total number of departures, Ndep
# - integral of the number of client in time
# - store the time of the last event (for computing the integral)
# - total delay in the queue 
# ******************************************************************************
class Measure:
    def __init__(self,Narr,Ndep,NAveraegUser,OldTimeEvent,AverageDelay):
        self.arr = Narr
        self.dep = Ndep
        self.ut = NAveraegUser
        self.oldT = OldTimeEvent
        self.delay = AverageDelay
        
# ******************************************************************************
# Client
# 
# Identify the client with
# - type: for future use
# - time of arrival (for computing the delay, i.e., time in the queue)
# ******************************************************************************
class Client:
    def __init__(self,type,arrival_time):
        self.type = type
        self.arrival_time = arrival_time


# ******************************************************************************
# ARRIVAL: event function
# 
# Receive in input 
# - the FES, for possibly schedule new events
# - the queue of the clients
# ******************************************************************************
def arrival(time, FES, queue):
    global users
            
    # cumulate statistics
    data.arr += 1
    data.ut += users*(time-data.oldT)
    data.oldT = time

    # sample the time until the next arrival
    inter_arrival = random.expovariate(lambd=1.0/ARRIVAL)
    
    # schedule the next arrival
    FES.put((time + inter_arrival, "arrival"))

    # update the state variable, by increasing the no. of clients by 1
    users += 1
    
    # create a record for the client
    client = Client(TYPE1,time)

    # insert the record in the queue
    queue.append(client)

    # if the server is idle start the service
    if users==1:
        # sample the service time
        service_time = random.expovariate(1.0/SERVICE)
        # schedule the departure of the client
        FES.put((time + service_time, "departure"))


# ******************************************************************************
def departure(time, FES, queue):
    global users

    # get the first element from the queue
    client = queue.pop(0)
        
    # cumulate statistics
    data.dep += 1
    data.ut += users*(time-data.oldT)
    data.oldT = time
    data.delay += (time-client.arrival_time)


    # update the state variable, by decreasing the no. of clients by 1
    users -= 1
    
    # check whether there are more clients to in the queue
    if users >0:
        # sample the service time
        service_time = random.expovariate(1.0/SERVICE)
        # schedule the departure of the client
        FES.put((time + service_time, "departure"))

        
# ******************************************************************************
# Initialization
# ******************************************************************************
#


#arrivals=0
# State variable: number of users
users=0  
# the simulation time 
time = 0
# Queue of the clients
queue=[]  
# Collect measurements
data = Measure(0,0,0,0,0)
# Future Event Set: the list of events in the form: (time, type)
FES = PriorityQueue()
# schedule the first arrival at t=0
FES.put((0, "arrival"))
# Initialize the random number generator    
random.seed(42)

        
# ******************************************************************************
# Event-loop 
# ******************************************************************************
#


# Simulate until the simulated time reaches a constant
while time < SIM_TIME:
    # Extract next event from the FES
    (time, event_type) = FES.get()
    # Call the event functions based on the event type 
    if event_type == "arrival":
        arrival(time, FES, queue)

    elif event_type == "departure":
        departure(time, FES, queue)


# ******************************************************************************
# Print outputs
# ******************************************************************************


print("\n\n\n","*"*10,"  MEASUREMENTS  ","*"*10,"\n")
print("No. of users in the queue at the end of the simulation:",users,\
      "\nTot. no. of arrivals =",data.arr,"- Tot. no. of departures =",data.dep)
print("Actual queue size: ",len(queue))
if len(queue)>0:
    print("Arrival time of the last element in the queue:",queue[len(queue)-1].arrival_time)

print("\n\nLoad: ",LOAD)
print("Nominal arrival rate: ",1.0/ARRIVAL)
print("Measured arrival rate",data.arr/time,"\nMeasured departure rate: ",data.dep/time)
theorical=(1.0/ARRIVAL)/(1.0/SERVICE-1.0/ARRIVAL)
print("\n\nAverage number of users\nTheorical: ", theorical,\
      "  -  Empirical: ",data.ut/time)

theorical=1.0/(1.0/SERVICE-1.0/ARRIVAL)
print("Average delay \nTheorical= ",theorical,"  -  Empirical: ",data.delay/data.dep)

print("\n","*"*40)
    




 **********   MEASUREMENTS   ********** 

No. of users in the queue at the end of the simulation: 4 
Tot. no. of arrivals = 42253 - Tot. no. of departures = 42249
Actual queue size:  4
Arrival time of the last element in the queue: 499996.1379328808


Load:  0.85
Nominal arrival rate:  0.08499999999999999
Measured arrival rate 0.08450408528590261 
Measured departure rate:  0.08449608546716444


Average number of users
Theorical:  5.666666666666661   -  Empirical:  5.9530212067994865
Average delay 
Theorical=  66.66666666666661   -  Empirical:  70.45104137956679

 ****************************************
